### Config

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
from huggingface_hub import hf_hub_download
import torch
from uni2ts.eval_util.plot import plot_single, plot_next_multi
from uni2ts.model.moirai import MoiraiForecast
from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.split import split

In [28]:
year = pd.read_parquet('/Users/tomaltenborg/Documents/Master/Master thesis/Notebooks/M3 Data/M3_yearly_processed.parquet')
year = year[year['Series'] == 1]
year

,Series,N,NF,Category,Value,Date
0,1,20,6,MICRO,940.66,1975-01-01
1,1,20,6,MICRO,1084.86,1976-01-01
2,1,20,6,MICRO,1244.98,1977-01-01
3,1,20,6,MICRO,1445.02,1978-01-01
4,1,20,6,MICRO,1683.17,1979-01-01
5,1,20,6,MICRO,2038.15,1980-01-01
6,1,20,6,MICRO,2342.52,1981-01-01
7,1,20,6,MICRO,2602.45,1982-01-01
8,1,20,6,MICRO,2927.87,1983-01-01
9,1,20,6,MICRO,3103.96,1984-01-01


### Test to see how the model functions

In [41]:
# Parameters (you might need to adjust these)
SIZE = "small"  # Choose from {'small', 'base', 'large'}
PDT = 6  # Prediction length: set to the maximum NF in your data if it varies
CTX = 200  # Context length: typically larger than PDT
PSZ = "auto"  # Patch size
BSZ = 1  # Batch size: 1 for simplicity in iterating forecasts

In [29]:
def prepare_data(df):
    train_data = {}
    test_data = {}
    
    # Group by 'Series' and then create training and testing datasets
    for series_id, group in df.groupby('Series'):
        N = group['N'].iloc[0]  # Assuming all rows in a group have the same N
        NF = group['NF'].iloc[0]  # Assuming all rows in a group have the same NF
        
        # Sorting by date to ensure correct temporal order
        group = group.sort_values('Date')
        
        # Splitting the data
        train_idx = N - NF
        train_data[series_id] = group.iloc[:train_idx]
        test_data[series_id] = group.iloc[train_idx:N]  # This should contain NF elements

    return train_data, test_data

# Prepare the datasets
train_data, test_data = prepare_data(year)


In [31]:
# Load the MOIRAI model
# Updated script to load the model with additional required parameters
model = MoiraiForecast.load_from_checkpoint(
    checkpoint_path=model_checkpoint,
    prediction_length=PDT,
    context_length=CTX,
    patch_size=PSZ,
    num_samples=100,  # Number of sample paths for probabilistic forecasts
    target_dim=1,  # Adjust this if your target variable has more than one dimension
    feat_dynamic_real_dim=0,  # Adjust based on the number of dynamic real-valued features you have
    past_feat_dynamic_real_dim=0,  # Adjust based on the number of past dynamic real-valued features you have
    map_location="cuda:0" if torch.cuda.is_available() else "cpu",  # Adjust depending on whether you're using a GPU
)

# Proceed with the rest of the script as previously discussed
predictor = model.create_predictor(batch_size=BSZ)


In [32]:
def forecast_series(predictor, train_data, test_data):
    forecasts = {}
    
    for series_id, data in train_data.items():
        # Assuming 'Value' column holds the time series values
        test_input = data['Value'].values
        forecast = predictor.predict(test_input)
        forecasts[series_id] = forecast
    
    return forecasts

# Generate forecasts
forecasts = forecast_series(predictor, train_data, test_data)


In [42]:
test1 = iter(forecasts)
test2 = next(test1)
test2

1

### From Salesforce GitHub

In [105]:
year = pd.read_parquet('/Users/tomaltenborg/Documents/Master/Master thesis/Notebooks/M3 Data/M3_yearly_processed.parquet')
year = year[['Series', 'Date', 'Value']]
# year = year.loc[year['Series'] == 1]
year.set_index('Date', inplace=True)

In [106]:
# year = year.loc[year['Series'] == 1]
# year

In [107]:
# Convert into GluonTS dataset
ds1 = PandasDataset.from_long_dataframe(year, target="Value", item_id="Series")

In [130]:
# Parameters (you might need to adjust these)
SIZE = "base"  # Choose from {'small', 'base', 'large'}
PDT = 6  # Prediction length: set to the maximum NF in your data if it varies
CTX = 50  # Context length: typically larger than PDT
PSZ = "auto"  # Patch size
BSZ = 1  # Batch size: 1 for simplicity in iterating forecasts

In [138]:
year_try = year.loc[(year['Series'] == 1) & (year['Series'] == 2) & (year['Series'] == 3)]

In [139]:
ls = []
for series in year_try['Series'].unique():
    # Prepare data
    df = year_try.loc[year_try['Series'] == series]
    # ds = PandasDataset.from_long_dataframe(df, target="Value", item_id="Series")
    ds = PandasDataset(dict(df))
    # Prepare model
    model = MoiraiForecast.load_from_checkpoint(
        checkpoint_path=hf_hub_download(
            repo_id=f"Salesforce/moirai-1.0-R-{SIZE}", filename="model.ckpt"
        ),
        prediction_length=PDT,
        context_length=CTX,
        patch_size=PSZ,
        num_samples=15,
        target_dim=1,
        feat_dynamic_real_dim=ds.num_feat_dynamic_real,
        past_feat_dynamic_real_dim=ds.num_past_feat_dynamic_real,
        map_location="cuda:0" if torch.cuda.is_available() else "cpu",
    )

    predictor = model.create_predictor(batch_size=BSZ)
    forecasts = predictor.predict(ds.input)
    print(series)
    # input_it = iter(test_data.input)
    # label_it = iter(test_data.label)
    forecast_it = iter(forecasts)
    forecast = next(forecast_it)
    ls.append(forecast)

In [135]:
ls

[]

In [136]:
try:
    prediction = next(forecast_it)
    print(prediction)
except StopIteration:
    print("No more predictions.")


No more predictions.
